## Load Data

In [1]:
import pandas as pd
import random
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

#train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Class', 'TIMESTAMP'])
train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Quality', 'TIMESTAMP'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

In [2]:
train_x.shape

(598, 2878)

## Preprocessing

In [3]:
from sklearn.preprocessing import LabelEncoder

qual_col = ['LINE','PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    
    test_x[i] = le.transform(test_x[i])

print('done')

done


In [4]:
train_x.shape

(598, 2878)

### split

In [5]:
## PRODUCT_CODE

# train
train_x_1 = train_x[train_x['PRODUCT_CODE'] == 0].drop('Y_Class', axis=1)
train_x_2 = train_x[train_x['PRODUCT_CODE'] == 1].drop('Y_Class', axis=1)
train_x_3 = train_x[train_x['PRODUCT_CODE'] == 2].drop('Y_Class', axis=1)

train_y_1 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 0]
train_y_2 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 1]
train_y_3 = train_x['Y_Class'][train_x['PRODUCT_CODE'] == 2]

print('train_x_1 shape :', train_x_1.shape,
      '\ntrain_x_2 shape :', train_x_2.shape,
      '\ntrain_x_3 shape :', train_x_3.shape)

# test
test_x_1 = test_x[test_x['PRODUCT_CODE'] == 0]
test_x_2 = test_x[test_x['PRODUCT_CODE'] == 1]
test_x_3 = test_x[test_x['PRODUCT_CODE'] == 2]

print('done')

train_x_1 shape : (249, 2877) 
train_x_2 shape : (6, 2877) 
train_x_3 shape : (343, 2877)
done


In [6]:
train_x_1['LINE'].value_counts()

2    78
1    70
0    59
3    42
Name: LINE, dtype: int64

In [7]:
# LINE

## TRAIN

# line 1
train_x_1_1 = train_x_1[train_x_1['LINE'] == 0]

# line 2
train_x_1_2 = train_x_1[train_x_1['LINE'] == 1]

# line 3
train_x_1_3 = train_x_1[train_x_1['LINE'] == 2]

# line 4
train_x_1_4 = train_x_1[train_x_1['LINE'] == 3]

# line 5
train_x_2_5 = train_x_2[train_x_2['LINE'] == 4]
train_x_3_5 = train_x_3[train_x_3['LINE'] == 4]

# line 6
train_x_2_6 = train_x_2[train_x_2['LINE'] == 5]
train_x_3_6 = train_x_3[train_x_3['LINE'] == 5]

train_set = [train_x_1_1, train_x_1_2, train_x_1_3, train_x_1_4, train_x_2_5, train_x_3_5, train_x_2_6, train_x_3_6]

for set in train_set:
    print(set.shape)

(59, 2877)
(70, 2877)
(78, 2877)
(42, 2877)
(3, 2877)
(172, 2877)
(3, 2877)
(171, 2877)


In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

train_scale = []

for train in train_set:
    scaler.fit_transform(train)
    train = pd.DataFrame(data=scaler.fit_transform(train), columns=train.columns, index=train.index)
    train_scale.append(train)


for set in train_scale:
    print(set.shape)

(59, 2877)
(70, 2877)
(78, 2877)
(42, 2877)
(3, 2877)
(172, 2877)
(3, 2877)
(171, 2877)


In [9]:
train_x_scale = pd.concat(train_scale, axis=0).sort_index()

train_x_scale.shape

(598, 2877)

In [10]:
from sklearn.model_selection import train_test_split

x_t, x_v, y_t, y_v = train_test_split(train_x_scale, train_y, test_size=0.2, random_state=42)

## Model

XGBClassifier

In [11]:
from xgboost import XGBClassifier # 회귀트리

xgb = XGBClassifier()
xgb.fit(x_t, y_t)
pred = xgb.predict(x_v)

from sklearn.metrics import accuracy_score

accuracy_score(pred, y_v)

0.7166666666666667

In [14]:
from xgboost import XGBClassifier # 회귀트리

xgb = XGBClassifier()
xgb.fit(train_x_scale, train_y)
pred = xgb.predict(test_x)


submit_csv = pd.read_csv('./sample_submission.csv')
submit_csv['Y_Class'] = pred
submit_csv.to_csv('XGBoost_null_sacle_submission.csv', index=False)